In [1]:
import pandas as pd
from build_database import CocktailClient

In [2]:
london = pd.read_csv("data/london_transactions.csv.gz", compression="gzip", sep=r"\t", names=['datetime','drink','price'], index_col=0)
budapest = pd.read_csv("data/budapest.csv.gz", compression="gzip",names=['datetime','drink','price'], index_col=0, header=0)
ny = pd.read_csv("data/ny.csv.gz", compression="gzip",names=['datetime','drink','price'], index_col=0, header=0)

C:\Users\alast\AppData\Local\Temp\ipykernel_628\3018081244.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  london = pd.read_csv("data/london_transactions.csv.gz", compression="gzip", sep=r"\t", names=['datetime','drink','price'], index_col=0)


In [3]:
london['datetime'] = london['datetime'].apply(pd.to_datetime)
budapest['datetime'] = budapest['datetime'].apply(pd.to_datetime)
ny['datetime'] = ny['datetime'].apply(pd.to_datetime)

In [22]:
london.head(5)

,datetime,drink,price
0,2020-12-30 13:17:00,Mojito,5.5
1,2020-12-27 11:06:15,Sidecar,11.0
2,2020-12-25 17:40:36,Snowball,11.0
3,2020-12-28 08:55:02,Limeade,3.5
4,2020-12-30 20:10:03,Limeade,3.5


In [7]:
london['drink'].value_counts()

drink
Limeade                       3318
Snowball                      2991
Orangeade                     1831
Bellini                       1786
Raspberry Cooler              1566
                              ... 
Melya                           16
Penicillin                      15
Lemon Elderflower Spritzer      14
Popped cherry                    3
Royal Flush                      1
Name: count, Length: 230, dtype: int64

In [18]:
london.shape

(50000, 3)

In [23]:
budapest.head(5)

,datetime,drink,price
0,2020-12-30 15:15:53,Sweet Sangria,4.0
1,2020-12-31 18:02:26,Grim Reaper,4.0
2,2020-12-30 00:01:27,Kool-Aid Slammer,3.0
3,2020-12-26 21:58:30,Margarita,7.5
4,2020-12-27 20:51:36,Margarita,7.5


In [28]:
budapest['drink'].value_counts()

drink
Hawaiian Cocktail    2592
Imperial Cocktail    2511
Margarita            2453
Long Island Tea      1784
Manhattan            1397
                     ... 
Melya                  10
Penicillin              9
Royal Flush             6
Whiskey Sour            5
Popped cherry           2
Name: count, Length: 232, dtype: int64

In [23]:
budapest.shape

(50000, 3)

In [24]:
ny.head(5)

,datetime,drink,price
0,2020-12-26 22:47:00,Paradise,4.20
1,2020-12-26 08:34:00,Dry Martini,8.00
2,2020-12-31 18:41:00,Kill the cold Smoothie,11.00
3,2020-12-29 15:38:00,Cuba Libra,2.99
4,2020-12-26 07:09:00,Campari Beer,7.50


In [9]:
ny['drink'].value_counts()

drink
Cuba Libra       2322
Chicago Fizz     2232
Pina Colada      1832
Dry Martini      1826
Kurant Tea        417
                 ... 
Queen Bee          13
Caipirissima       13
Whiskey Sour        5
Royal Flush         3
Popped cherry       1
Name: count, Length: 231, dtype: int64

In [27]:
ny.shape

(50000, 3)

Need to take care around column names, separators and indexes as well as the format of the date time columns

Need a dimension table for the glass types, bar id and drinks combo bar as price may be different for a drink across the bars

In [3]:
cocktail_client = CocktailClient()

In [10]:
lst = cocktail_client.list_all_cocktails("a").json()

In [15]:
lst['drinks'][0]

{'idDrink': '17222',
 'strDrink': 'A1',
 'strDrinkAlternate': None,
 'strTags': None,
 'strVideo': None,
 'strCategory': 'Cocktail',
 'strIBA': None,
 'strAlcoholic': 'Alcoholic',
 'strGlass': 'Cocktail glass',
 'strInstructions': 'Pour all ingredients into a cocktail shaker, mix and serve over ice into a chilled glass.',
 'strInstructionsES': 'Vierta todos los ingredientes en una coctelera, mezcle y sirva con hielo en un vaso frío.',
 'strInstructionsDE': 'Alle Zutaten in einen Cocktailshaker geben, mischen und über Eis in ein gekühltes Glas servieren.',
 'strInstructionsFR': None,
 'strInstructionsIT': 'Versare tutti gli ingredienti in uno shaker, mescolare e servire con ghiaccio in un bicchiere freddo.',
 'strInstructionsZH-HANS': None,
 'strInstructionsZH-HANT': None,
 'strDrinkThumb': 'https://www.thecocktaildb.com/images/media/drink/2x8thr1504816928.jpg',
 'strIngredient1': 'Gin',
 'strIngredient2': 'Grand Marnier',
 'strIngredient3': 'Lemon Juice',
 'strIngredient4': 'Grenadine'

In [3]:
marg = cocktail_client.get_info_for_cocktail_name("margarita")

In [4]:
marg.json()

{'drinks': [{'idDrink': '11007',
   'strDrink': 'Margarita',
   'strDrinkAlternate': None,
   'strTags': 'IBA,ContemporaryClassic',
   'strVideo': None,
   'strCategory': 'Ordinary Drink',
   'strIBA': 'Contemporary Classics',
   'strAlcoholic': 'Alcoholic',
   'strGlass': 'Cocktail glass',
   'strInstructions': 'Rub the rim of the glass with the lime slice to make the salt stick to it. Take care to moisten only the outer rim and sprinkle the salt on it. The salt should present to the lips of the imbiber and never mix into the cocktail. Shake the other ingredients with ice, then carefully pour into the glass.',
   'strInstructionsES': None,
   'strInstructionsDE': 'Reiben Sie den Rand des Glases mit der Limettenscheibe, damit das Salz daran haftet. Achten Sie darauf, dass nur der äußere Rand angefeuchtet wird und streuen Sie das Salz darauf. Das Salz sollte sich auf den Lippen des Genießers befinden und niemals in den Cocktail einmischen. Die anderen Zutaten mit Eis schütteln und vorsi

In [6]:
len(marg.json()['drinks'])

6

In [9]:
for i in marg.json()['drinks']:
    print (i['strDrink'])

Margarita
Blue Margarita
Tommy's Margarita
Whitecap Margarita
Strawberry Margarita
Smashed Watermelon Margarita


In [4]:
bar_data = pd.read_csv("data/bar_data.csv")

In [18]:
bar_data.head(5)

,glass_type,stock,bar
0,cocktail glass,8,budapest
1,shot glass,31,budapest
2,martini glass,13,budapest
3,highball glass,37,budapest
4,collins glass,37,budapest


In [26]:
bar_data.shape

(93, 3)

In [60]:
bar_data['bar'].value_counts()

bar
budapest    31
london      31
new york    31
Name: count, dtype: int64

In [41]:
bar_data['glass_type'].value_counts()

glass_type
cocktail glass              3
pitcher                     3
pousse cafe glass           3
whiskey glass               3
white wine glass            3
nick and nora glass         3
jar                         3
copper mug                  3
brandy snifter              3
cordial glass               3
wine glass                  3
balloon glass               3
mason jar                   3
irish coffee cup            3
beer glass                  3
hurricane glass             3
shot glass                  3
pint glass                  3
beer mug                    3
coffee mug                  3
punch bowl                  3
coupe glass                 3
beer pilsner                3
margarita glass             3
champagne flute             3
whiskey sour glass          3
old-fashioned glass         3
collins glass               3
highball glass              3
martini glass               3
margarita/coupette glass    3
Name: count, dtype: int64

In [5]:
bar_data['glass_type'] = bar_data['glass_type'].replace('coper mug','copper mug')

In [6]:
glasses = bar_data.groupby("glass_type").max().reset_index()

In [7]:
glasses['ID'] = glasses.index

In [8]:
glasses.rename(columns = {'ID': 'GlassID', 'glass_type': 'GlassName'}, inplace=True)

In [15]:
glasses[['GlassID','GlassName']]

,GlassID,GlassName
0,0,balloon glass
1,1,beer glass
2,2,beer mug
3,3,beer pilsner
4,4,brandy snifter
5,5,champagne flute
6,6,cocktail glass
7,7,coffee mug
8,8,collins glass
9,9,copper mug


In [10]:
bars = pd.DataFrame({"BarID": [1,2,3],"BarName": ["london","budapest","new york"]})

In [52]:
bar_stock = pd.merge(bar_data, bars, left_on='bar', right_on='BarName')
bar_glasses_stock = pd.merge(bar_stock, glasses, left_on="glass_type", right_on='GlassName')

In [53]:
bar_glasses_stock

,glass_type,stock_x,bar_x,BarID,BarName,GlassName,stock_y,bar_y,GlassID
0,cocktail glass,8,budapest,2,budapest,cocktail glass,8,new york,6
1,cocktail glass,42,london,1,london,cocktail glass,8,new york,6
2,cocktail glass,211,new york,3,new york,cocktail glass,8,new york,6
3,shot glass,31,budapest,2,budapest,shot glass,33,new york,26
4,shot glass,33,london,1,london,shot glass,33,new york,26
...,...,...,...,...,...,...,...,...,...
88,pousse cafe glass,43,london,1,london,pousse cafe glass,43,new york,24
89,pousse cafe glass,34,new york,3,new york,pousse cafe glass,43,new york,24
90,margarita/coupette glass,35,budapest,2,budapest,margarita/coupette glass,44,new york,17
91,margarita/coupette glass,44,london,1,london,margarita/coupette glass,44,new york,17


In [51]:
bar_stock

,glass_type,stock,bar,BarID,BarName
0,cocktail glass,8,budapest,2,budapest
1,shot glass,31,budapest,2,budapest
2,martini glass,13,budapest,2,budapest
3,highball glass,37,budapest,2,budapest
4,collins glass,37,budapest,2,budapest
...,...,...,...,...,...
88,nick and nora glass,33,new york,3,new york
89,white wine glass,39,new york,3,new york
90,whiskey glass,47,new york,3,new york
91,pousse cafe glass,34,new york,3,new york


In [11]:
bars

,BarID,BarName
0,1,london
1,2,budapest
2,3,new york


In [42]:
london_price = london[['drink','price']].groupby("drink").first()
london_price['BarID'] = 1
london_price.reset_index(inplace=True)
budapest_price = budapest[['drink','price']].groupby("drink").first()
budapest_price['BarID'] = 2
budapest_price.reset_index(inplace= True)
frames =[london_price,budapest_price]

In [43]:
all_price =pd.concat(frames)

In [44]:
all_price

,drink,price,BarID
0,A. J.,11.0,1
1,Ace,12.0,1
2,Adam,3.5,1
3,Addison,12.0,1
4,Affair,8.0,1
...,...,...,...
227,Zimadori Zinger,12.0,2
228,Zippy's Revenge,5.0,2
229,Zizi Coin-coin,5.0,2
230,Zoksel,6.5,2


In [7]:
all_drinks = pd.DataFrame(set(london['drink'].unique()) | set(budapest['drink'].unique()) | set(ny['drink'].unique()), columns=["DrinkName"])

In [14]:
all_drinks['DrinkID'] = all_drinks.index

In [15]:
all_drinks

,DrinkName,DrinkID
0,Royal Gin Fizz,0
1,Zorbatini,1
2,Kill the cold Smoothie,2
3,Egg Cream,3
4,Scotch Sour,4
...,...,...
233,Godchild,233
234,Dry Rob Roy,234
235,Hemingway Special,235
236,Grizzly Bear,236


In [13]:
def return_glass(cocktail_client, drink_name):
    res = cocktail_client.get_info_for_cocktail_name(drink_name).json()
    return (res['drinks'][0]['strGlass'])

In [14]:
all_drinks = pd.DataFrame(set(london['drink'].unique()) | set(budapest['drink'].unique()) | set(ny['drink'].unique()), columns=["DrinkName"])
all_drinks['DrinkID'] = all_drinks.index
all_drinks["Glass"] = all_drinks.apply(lambda row: return_glass(cocktail_client, row['DrinkName']), axis=1)

In [22]:
all_drinks['Glass'] = all_drinks['Glass'].apply(lambda row: row.lower())


In [25]:
all_drinks

,DrinkName,DrinkID,Glass
0,Kool First Aid,0,shot glass
1,Zippy's Revenge,1,old-fashioned glass
2,Cuba Libre,2,highball glass
3,Addison,3,martini glass
4,Karsk,4,highball glass
...,...,...,...
233,Penicillin,233,old-fashioned glass
234,Victory Collins,234,collins glass
235,Grand Blue,235,old-fashioned glass
236,Limona Corona,236,beer glass


In [29]:
drinks_glass = pd.merge(all_drinks, glasses[['GlassID','GlassName']], left_on='Glass', right_on='GlassName')

In [30]:
drinks_glass

,DrinkName,DrinkID,Glass,GlassID,GlassName
0,Kool First Aid,0,shot glass,26,shot glass
1,Chocolate Milk,11,shot glass,26,shot glass
2,Big Red,21,shot glass,26,shot glass
3,Bob Marley,27,shot glass,26,shot glass
4,Orange Crush,42,shot glass,26,shot glass
...,...,...,...,...,...
233,Death in the Afternoon,203,margarita glass,16,margarita glass
234,Barracuda,212,margarita glass,16,margarita glass
235,The Philosopher,221,margarita glass,16,margarita glass
236,Ipamena,153,wine glass,30,wine glass


In [45]:
all_drinks

,DrinkName,DrinkID,Glass
0,Kool First Aid,0,shot glass
1,Zippy's Revenge,1,old-fashioned glass
2,Cuba Libre,2,highball glass
3,Addison,3,martini glass
4,Karsk,4,highball glass
...,...,...,...
233,Penicillin,233,old-fashioned glass
234,Victory Collins,234,collins glass
235,Grand Blue,235,old-fashioned glass
236,Limona Corona,236,beer glass


In [46]:
all_price

,drink,price,BarID
0,A. J.,11.0,1
1,Ace,12.0,1
2,Adam,3.5,1
3,Addison,12.0,1
4,Affair,8.0,1
...,...,...,...
227,Zimadori Zinger,12.0,2
228,Zippy's Revenge,5.0,2
229,Zizi Coin-coin,5.0,2
230,Zoksel,6.5,2


In [47]:
price_drinks = pd.merge(all_price, all_drinks, left_on="drink", right_on = "DrinkName")

In [48]:
price_drinks

,drink,price,BarID,DrinkName,DrinkID,Glass
0,A. J.,11.00,1,A. J.,197,cocktail glass
1,A. J.,11.00,2,A. J.,197,cocktail glass
2,Ace,12.00,1,Ace,222,martini glass
3,Ace,12.00,2,Ace,222,martini glass
4,Adam,3.50,1,Adam,215,cocktail glass
...,...,...,...,...,...,...
457,Imperial Cocktail,4.99,2,Imperial Cocktail,38,cocktail glass
458,Long Island Tea,8.00,2,Long Island Tea,214,highball glass
459,Manhattan,4.20,2,Manhattan,96,cocktail glass
460,Margarita,7.50,2,Margarita,179,cocktail glass
